# Syncronization of Microscope frame to camera frames


Loading the Camera csv(video_timestamp) and converting into a data frame in milliseconds


In [1]:
# Importing the packages

import pandas as pd
import numpy as np

In [2]:
# Load the data files
camera_df = pd.read_csv('camera.csv',sep=r'\s+', names= ['hrs','min','sec','mill'])
microscope_df = pd.read_csv('sync_2019-09-30T13_29_11.csv',sep=r'\s+',names= ['hrs','min','sec','mill','frame','frame_no'])

In [3]:
camera_df.head(2)

,hrs,min,sec,mill
0,13,28,0,457
1,13,28,0,464


In [4]:
camera_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55151 entries, 0 to 55150
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   hrs     55151 non-null  int64
 1   min     55151 non-null  int64
 2   sec     55151 non-null  int64
 3   mill    55151 non-null  int64
dtypes: int64(4)
memory usage: 1.7 MB


In [5]:
microscope_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27012 entries, 0 to 27011
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   hrs       27012 non-null  int64 
 1   min       27012 non-null  int64 
 2   sec       27012 non-null  int64 
 3   mill      27012 non-null  int64 
 4   frame     27012 non-null  object
 5   frame_no  27012 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 1.2+ MB


In [6]:
microscope_df.head(2)

,hrs,min,sec,mill,frame,frame_no
0,13,29,32,989,"Fra,",2318121272
1,13,29,33,22,"Fra,",2318121305


In [7]:
# Convert time hr:min:sec:mill to milliseconds and creating a new column with millisec

def convert_to_ms(df):
   
    """ Convert hrs, min, sec, mill columns to total milliseconds and create a new column 'time_ms'. """
    return (df['hrs']*3600000 + df['min']*60000 + df['sec']*1000 + df['mill'])
   

In [8]:
camera_df['camera_time_in_ms'] = convert_to_ms(camera_df)
microscope_df['microscope_time_in_ms'] = convert_to_ms(microscope_df)

In [9]:
camera_df.head(2)

,hrs,min,sec,mill,camera_time_in_ms
0,13,28,0,457,48480457
1,13,28,0,464,48480464


In [10]:
camera_df.isnull().sum()

hrs                  0
min                  0
sec                  0
mill                 0
camera_time_in_ms    0
dtype: int64

In [11]:
microscope_df.isnull().sum()

hrs                      0
min                      0
sec                      0
mill                     0
frame                    0
frame_no                 0
microscope_time_in_ms    0
dtype: int64

In [12]:
camera_df[camera_df.duplicated(subset=['hrs','min','sec','mill','camera_time_in_ms'])]

,hrs,min,sec,mill,camera_time_in_ms


In [13]:
microscope_df[microscope_df.duplicated(subset=['hrs','min','sec','mill','frame','frame_no','microscope_time_in_ms'])]

,hrs,min,sec,mill,frame,frame_no,microscope_time_in_ms


# Observations and insights

No dupicates found in two files and no null values were found

Deleting the hr, min, sec, mill columns and keeping only millisecond column

In [14]:
camera_df.drop(columns=['hrs','min','sec','mill'], inplace =True)

In [15]:
microscope_df.drop(columns=['hrs','min','sec','mill','frame'],inplace=True)

In [16]:
microscope_df.head(3)

,frame_no,microscope_time_in_ms
0,2318121272,48572989
1,2318121305,48573022
2,2318121338,48573054


In [17]:
# Reordering the columns in micorscope_df

microscope_df = microscope_df[['microscope_time_in_ms','frame_no']]

In [18]:
microscope_df.head(3)

,microscope_time_in_ms,frame_no
0,48572989,2318121272
1,48573022,2318121305
2,48573054,2318121338


In [19]:
camera_df.head()

,camera_time_in_ms
0,48480457
1,48480464
2,48480479
3,48480496
4,48480512


In [20]:
microscope_df.tail()

,microscope_time_in_ms,frame_no
27007,49473686,2319021927
27008,49473719,2319021960
27009,49473752,2319021993
27010,49473785,2319022027
27011,49473822,2319022060


In [21]:
camera_df.tail()

,camera_time_in_ms
55146,49399783
55147,49399800
55148,49399817
55149,49399834
55150,49399850


In [22]:
# Ensuring both were sorted by time

camera_df = camera_df.sort_values('camera_time_in_ms').reset_index(drop=True)
microscope_df = microscope_df.sort_values('microscope_time_in_ms').reset_index(drop = True)

In [23]:
def synchronize_two_to_one(camera_df, microscope_df):
    """
    Synchronize 2 camera frames to 1 microscope frame.
    Camera always leads → only take camera frames before or equal to each microscope frame to reduce jetter effect
    
    """
    synced_data = []
    camera_used = set()
    expected_ratio = 2

    for i, micro_time in enumerate(microscope_df['microscope_time_in_ms']):
        # Filter camera frames before or equal to microscope timestamp
        valid_cams = camera_df[camera_df['camera_time_in_ms'] <= micro_time]

        if len(valid_cams) < expected_ratio:
            continue  # skip if not enough camera frames before microscope frame

        # Take the last 2 camera frames before microscope frame
        mapped_cams = valid_cams.tail(expected_ratio)['camera_time_in_ms'].tolist()

        synced_data.append({
            'microscope_frame_no': microscope_df.loc[i, 'frame_no'],
            'microscope_time_in_ms': micro_time,
            'camera_times': mapped_cams
        })

        # Mark used frames
        camera_used.update(valid_cams.tail(expected_ratio).index.tolist())

    synced_df = pd.DataFrame(synced_data)
    unused_cameras = camera_df.drop(index=list(camera_used)).reset_index(drop=True)

    return synced_df, unused_cameras

In [24]:
# --- Perform synchronization ---
final_synced_df, unused_camera_frames = synchronize_two_to_one(camera_df, microscope_df)


In [25]:
final_synced_df.head(20)

,microscope_frame_no,microscope_time_in_ms,camera_times
0,2318121272,48572989,"[48572968, 48572984]"
1,2318121305,48573022,"[48573001, 48573018]"
2,2318121338,48573054,"[48573034, 48573051]"
3,2318121371,48573087,"[48573067, 48573084]"
4,2318121405,48573120,"[48573101, 48573118]"
5,2318121438,48573153,"[48573135, 48573152]"
6,2318121472,48573189,"[48573168, 48573185]"
7,2318121505,48573222,"[48573201, 48573218]"
8,2318121538,48573255,"[48573234, 48573251]"
9,2318121572,48573288,"[48573269, 48573284]"


In [26]:
final_synced_df.tail()

,microscope_frame_no,microscope_time_in_ms,camera_times
27007,2319021927,49473686,"[49399834, 49399850]"
27008,2319021960,49473719,"[49399834, 49399850]"
27009,2319021993,49473752,"[49399834, 49399850]"
27010,2319022027,49473785,"[49399834, 49399850]"
27011,2319022060,49473822,"[49399834, 49399850]"


In [29]:
unused_camera_frames.head()


,camera_time_in_ms
0,48480457
1,48480464
2,48480479
3,48480496
4,48480512


In [28]:
# to convert the final dataframe to csv file
final_synced_df.to_csv('final_sync.csv', index=False, header=False)  


# Observations and Insights

1.Successfully synchronized the camera and microscope time frames using a 1-to-2 mapping, considering the camera frame rate is approximately double the microscope frame rate.

2.To minimize jitter effects, mapping was done in one consistent direction  (from microscope to camera frames).

3.Unused camera frames were tracked separately in a dedicated DataFrame for further analysis.